In [3]:
import pandas as pd

In [15]:
df = pd.read_html('./data/2001Y_1M.xls',encoding='utf-8')
df[0]

,0,1
0,NaN,﻿<html><head runat='server'><meta http-equiv=C...


In [25]:
# bb = pd.ExcelWriter('out.xlsx')
# df[0].to_excel(bb)
# bb.close()